In [2]:
import numpy as np
import torch
from kan import KAN
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_error

In [3]:
# Устройство: если есть GPU, используем его, иначе CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем: {device} — надеюсь, это не старый калькулятор!")

Используем: cpu — надеюсь, это не старый калькулятор!


In [4]:
# Определите сложную функцию для подгонки
def f(x, y):
    return np.sin(2 * np.pi * x) * np.exp(-y**2) + np.cos(3 * np.pi * y) * x**2

# create dataset f(x,y) = exp(sin(pi*x)+y^2)
f= lambda x:torch.sin(2 * torch.pi * x) * torch.exp(-x[:,[1]]**2) + torch.cos(3 * torch.pi * x[:,[1]]) * x**2
dataset = create_dataset(f, n_var=2)
dataset['train_input'].shape, dataset['train_label'].shape

# Сгенерируйте данные
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

In [5]:
# Подготовьте данные для KAN (перевод в формат torch)
dataset = {}
dataset['train_input'] = torch.from_numpy(np.column_stack((X.ravel(), Y.ravel()))).float()
dataset['train_label'] = torch.from_numpy(Z.ravel()).float().unsqueeze(1)
dataset['test_input'] = dataset['train_input']  # Для простоты используем те же данные для теста
dataset['test_label'] = dataset['train_label']

In [6]:
# Инициализируйте модель KAN
model = KAN(width=[2, 10, 1], grid=10, k=3)  # Увеличим ширину и grid для сложной функции


checkpoint directory created: ./model
saving model version 0.0


In [7]:
# Обучите модель
results = model.train(dataset, opt="LBFGS", steps=50, lamb=0.01, lamb_entropy=10.)

# Сделайте предсказания
grid_pred = model(dataset['test_input']).detach().numpy().reshape(X.shape)


TypeError: Module.train() got an unexpected keyword argument 'opt'

In [ ]:
# Оцените модель
mse = mean_squared_error(dataset['test_label'].numpy(), model(dataset['test_input']).detach().numpy())
print(f"Mean Squared Error: {mse}")


In [ ]:
# Визуализация структуры модели KAN
plt.figure(figsize=(10, 5))
model.plot(beta=100)
plt.title("Структура модели KAN")
plt.show()

In [ ]:
# Визуализация результатов (3D-графики)
fig = plt.figure(figsize=(15, 5))

# Оригинальная функция
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
ax1.set_title("Оригинальная функция")
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.set_zlabel("f(x, y)")

# Предсказания KAN
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.plot_surface(X, Y, grid_pred, cmap='viridis', alpha=0.8)
ax2.set_title("Предсказания KAN")
ax2.set_xlabel("x")
ax2.set_ylabel("y")
ax2.set_zlabel("f(x, y)")

plt.show()



In [ ]:
# Визуализация разницы
plt.figure(figsize=(8, 6))
plt.contourf(X, Y, Z - grid_pred, levels=20, cmap='RdBu')
plt.colorbar(label="Разница (оригинал - предсказание)")
plt.title("Разница между оригинальной функцией и предсказаниями")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

# Получите и выведите символическую формулу
symbolic_formula = model.symbolic_formula()[0][0]
print("Символическая формула аппроксимации:", symbolic_formula)

In [ ]:
# Получите и выведите символическую формулу
symbolic_formula = model.symbolic_formula()[0][0]
print("Символическая формула аппроксимации:", symbolic_formula)